# Rede Neural Artificial

### Importando as bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

## Parte 1 - preprocessamento dos dados

### Importando o dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


### Codificando dados categóricos

Coluna 'gender'

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Coluna 'geography'

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Separando o dataset e o trainingset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Dimensionando os recursos

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Criando a  ANN (rede neural)

### Iniciando a ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adicionando a camada de entrada e a primeira camada oculta

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adicionando a segunda camada oculta

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Camada de saída

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Treinando a ANN

### Compilando a ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Trainando a ANN no Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32,epochs = 20)

Epoch 1/20
250/250 [==============================] - 0s 969us/step - loss: 0.6168 - accuracy: 0.6530
Epoch 2/20
250/250 [==============================] - 0s 887us/step - loss: 0.4722 - accuracy: 0.7961
Epoch 3/20
250/250 [==============================] - 0s 896us/step - loss: 0.4414 - accuracy: 0.7976
Epoch 4/20
250/250 [==============================] - 0s 1ms/step - loss: 0.4201 - accuracy: 0.8091
Epoch 5/20
250/250 [==============================] - 0s 882us/step - loss: 0.4006 - accuracy: 0.8232
Epoch 6/20
250/250 [==============================] - 0s 916us/step - loss: 0.3819 - accuracy: 0.8359
Epoch 7/20
250/250 [==============================] - 0s 914us/step - loss: 0.3682 - accuracy: 0.8456
Epoch 8/20
250/250 [==============================] - 0s 953us/step - loss: 0.3600 - accuracy: 0.8501
Epoch 9/20
250/250 [==============================] - 0s 927us/step - loss: 0.3550 - accuracy: 0.8539
Epoch 10/20
250/250 [==============================] - 0s 885us/step - loss: 0.3523 

## Part 4 - Previsões e análise do modelo

### Prever o resultado de uma única observação

**Pedido do banco**

Dados de certo cliente são enviados para nós e como cientista de dados do banco devemos dizer se ele vai ou não deixar a agência

País: França

Pontuação de crédito: 600

Gênero: Masculino

Idade: 40 anos

Tenure: 3 years

Saldo: \$ 60000

Nuúmero de Produtos: 2

Possui cartão de crédito? Sim

É um membro ativo? Sim

Salário estimado: \$ 50000


**Solução**

In [ ]:
#colocando todas as informaçoes do cliente que nos foram passadas:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.04419965]]


Nossa rede neural previu que existe 4,4% de chance do cliente deixar o banco.

### Prevendo resultados no testset

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Criando a Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1513   82]
 [ 192  213]]


0.863